In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [5]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))

for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)

hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)


tensor([[[-0.0558,  0.4515, -0.9094]],

        [[ 0.0729,  0.8562, -1.0990]],

        [[ 0.2927,  0.8433,  0.0486]],

        [[ 1.2114, -1.4602,  1.7995]],

        [[ 0.5176, -1.1685,  0.7591]]])
tensor([[[-0.0123,  0.0384,  0.3112]],

        [[ 0.0533,  0.1213,  0.2932]],

        [[ 0.0745,  0.1729,  0.3718]],

        [[-0.0109,  0.2084,  0.4683]],

        [[-0.0401,  0.2125,  0.4205]]])
(tensor([[[-0.0401,  0.2125,  0.4205]]]), tensor([[[-0.1283,  0.4025,  0.6685]]]))


In [7]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

{'the': 3, 'book': 8, 'Everybody': 5, 'that': 7, 'dog': 1, 'ate': 2, 'read': 6, 'The': 0, 'apple': 4}


In [9]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()
        
        print("sentence", sentence)
        print("tags", tags)
        
        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-1.0789, -1.1815, -1.0407],
        [-1.0654, -1.1625, -1.0709],
        [-1.0397, -1.1109, -1.1483],
        [-1.0906, -1.1236, -1.0821],
        [-1.0311, -1.2318, -1.0452]])
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence 

sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'th

sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'th

sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'th

sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'the', 'apple']
tags ['DET', 'NN', 'V', 'DET', 'NN']
sentence ['Everybody', 'read', 'that', 'book']
tags ['NN', 'V', 'DET', 'NN']
sentence ['The', 'dog', 'ate', 'th